# Defining Mongo Projections and Queries

Notice that there are a lot of fields for each object we obtain from MongoDB

In [45]:
from config import db

In [46]:
# So much junk
db.find_one()

{'_id': '556e0e24d6dfbb462880e9f7',
 'contributors': None,
 'coordinates': None,
 'created_at': 'Tue Jun 02 20:12:57 +0000 2015',
 'entities': {'hashtags': [],
  'symbols': [],
  'trends': [],
  'urls': [],
  'user_mentions': []},
 'favorite_count': 0,
 'favorited': False,
 'filter_level': 'low',
 'geo': None,
 'id': 605829475847176192,
 'id_str': '605829475847176192',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'lang': 'en',
 'place': {'attributes': {},
  'bounding_box': {'coordinates': [[[-95.823268, 29.522325],
     [-95.823268, 30.1546646],
     [-95.069705, 30.1546646],
     [-95.069705, 29.522325]]],
   'type': 'Polygon'},
  'country': 'United States',
  'country_code': 'US',
  'full_name': 'Houston, TX',
  'id': '1c69a67ad480e1b1',
  'name': 'Houston',
  'place_type': 'city',
  'url': 'https://api.twitter.com/1.1/geo/id/1c69a67ad480e1b1.json'},
 'possibly_se

We can remove a lot of the fields if we use a projection operator.

In [47]:
db.find_one(projection={"text":1})

{'_id': '556e0e24d6dfbb462880e9f7',
 'text': 'Snowcones &amp; Pickles w/ beer salt. Can I get an Amen 🙏 🙌'}

In [48]:
def project(*attributes, prefix=None):
    if not prefix:
        temp = "{}"
    else:
        temp = prefix + ".{}"
    return {temp.format(attr):1 for attr in attributes}

In [49]:
def union(*dtts):
    result = {}
    for d in dtts:
        result.update(d)
    return result

# Projections

These are used to select out the relevant features we want

In [61]:
proj_text = project(
    "text"
)

proj_time = project(
    "created_at"
)

proj_user = project(
    "friends_count",
    "followers_count",
    "statuses_count",
    "favourites_count", 
    "created_at",
    
    prefix="user"
)

proj_predict = project(
    "predict"
)

proj_label = project(
    "labels"
)

# Queries

In [62]:
# this finds everything that has an alch label

def exists(label, e=True):
    return {label: {"$exists": e}}

X = exists("labels")

In [63]:
db.find_one(X, projection=union(proj_label, proj_predict, proj_text, proj_time, proj_label, proj_user))

{'_id': '556e0ee3d6dfbb462880f0a5',
 'created_at': 'Tue Jun 02 20:16:08 +0000 2015',
 'labels': {'alcohol': 0},
 'predict': 0.5260502566296993,
 'text': 'Impatiently waiting to get our hands on the new @UrbanDecay Naked Smoky Palette. #CrueltyFree #VivaBeauty',
 'user': {'created_at': 'Thu Jun 12 22:14:05 +0000 2014',
  'favourites_count': 394,
  'followers_count': 407,
  'friends_count': 1997,
  'statuses_count': 823}}